# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [44]:
!pip install selenium

In [45]:
# chrome -> chromedriver
from selenium import webdriver

# PASSO A PASSO

# PASSO 1: ABRIR O NAVEGADOR
navegador = webdriver.Chrome()
navegador.get("https://www.google.com/")

In [46]:
# PASSO 2: IMPORTAR A BASE DE DADOS
import pandas as pd
tabela = pd.read_excel("commodities.xlsx")
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Trigo,1549.11,NaN,NaN
9,Tilápia,9.05,NaN,NaN


In [47]:
# .click() -> clicar
# .send.keys("texto") -> escrever texto
# .get_attribute() -> pega um atributo

# PARA SUBSITUIR TODOS CARACTERES ESPECIAIS DE UMA VEZ
# import unicodedata
# produto = unicodedata.normalize("NFKD", produto).encode("ascii", ignore")

# PASSO 3: PARA CADA PRODUTO NA BASE DE DADOS
# PASSO 4: PESQUISAR O PREÇO DO PRODUTO

for linha in tabela.index:
    produto = tabela.loc[linha, "Produto"]
    print(produto)
    produto = produto.replace("ó", "o").replace("ã", "a").replace("á", "a").replace("ç", "c").replace("ú", "u").replace("é", "e")
    
    link = f"https://www.melhorcambio.com/{produto}-hoje"
    navegador.get(link)

    preco = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
    preco = preco.replace(".", "").replace(",", ".")
    print(preco)
    
    tabela.loc[linha, "Preço Atual"] = float(preco)
    
display(tabela)

Milho
85.49
Soja
162.64
Boi
276.05
Petróleo
393.26
Algodão
498.88
Açúcar
134.64
Café
1087.87
Ouro
327.99
Trigo
1546.08
Tilápia
9.05


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,85.49,NaN
1,Soja,163.59,162.64,NaN
2,Boi,282.20,276.05,NaN
3,Petróleo,424.37,393.26,NaN
4,Algodão,497.76,498.88,NaN
5,Açúcar,136.23,134.64,NaN
6,Café,1092.87,1087.87,NaN
7,Ouro,321.77,327.99,NaN
8,Trigo,1549.11,1546.08,NaN
9,Tilápia,9.05,9.05,NaN


In [48]:
# PASSO 5: DECIDIR QUAIS PRODUTOS COMPRAR: PREENCHER A COLUNA DE COMPRAR
tabela["Comprar"] = tabela["Preço Atual"] < tabela["Preço Ideal"]
display(tabela)

# PASSO 6: EXPORTAR A BASE PARA O EXCEL
tabela.to_excel("commodities_atualizado.xlsx", index=False)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,85.49,False
1,Soja,163.59,162.64,True
2,Boi,282.20,276.05,True
3,Petróleo,424.37,393.26,True
4,Algodão,497.76,498.88,False
5,Açúcar,136.23,134.64,True
6,Café,1092.87,1087.87,True
7,Ouro,321.77,327.99,False
8,Trigo,1549.11,1546.08,True
9,Tilápia,9.05,9.05,False


In [49]:
navegador.quit()